In [77]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, confusion_matrix
from imblearn.over_sampling import RandomOverSampler
from num2words import num2words
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, KFold, RepeatedStratifiedKFold
from sklearn.metrics import f1_score, matthews_corrcoef, roc_auc_score
import word2number
from word2number import w2n
from sklearn.tree import DecisionTreeClassifier

In [78]:
def determine_covid_outcome(new_cases_per_100k, new_admits_per_100k, percent_beds_100k):
    if new_cases_per_100k < 200:
        if (new_admits_per_100k >= 10) | (percent_beds_100k > 10):
            if (new_admits_per_100k >= 20) | (percent_beds_100k >= 15):
                return "High"
            else:
                return "Medium"
        else:
            return "Low"
    elif 200 <= new_cases_per_100k:
        if (new_admits_per_100k >= 10) | (percent_beds_100k > 10):
            if (new_admits_per_100k >= 10) | (percent_beds_100k >= 10):
                return "High"
            else:
                return "Medium"




In [79]:
### this code it's ANY in the x week period 

def prep_training_test_data_period(data, proportion, no_weeks, weeks_in_futre, if_train, geography, weight_col):
## Get the weeks for the x and y datasets   
    x_weeks = []  
    y_weeks = []
    y_weeks_to_check = [] #check these weeks to see if any of them are equal to 1
    for week in no_weeks:
        test_week = int(week) + weeks_in_futre
        x_weeks.append('_' + num2words(week) + '_')
        for week_y in range(week+1, test_week+1):
                y_weeks_to_check.append('_' + num2words(week_y) + '_')
        y_weeks.append('_' + num2words(test_week) + '_')
    
## Divide up the test/train split
    #if is_geographic:
        # Calculate the index to start slicing from
    #    start_index = len(data['county']) // proportion[0] * proportion[1]
        # Divide up the dataset based on this proportion
    #    first_two_thirds = data['county'][:start_index]
    #    last_third = data['county'][start_index:]
    X_data = pd.DataFrame()
    y_data = pd.DataFrame()
    weights_all =  pd.DataFrame()
    missing_data = []
    ## Now get the training data 
    k = 0
    for x_week in x_weeks:
            y_week = y_weeks[k]
            k +=1

            weeks_x = [col for col in data.columns if x_week in col]
            columns_x  = [geography] + weeks_x + [weight_col]
            data_x = data[columns_x]

            weeks_y = [col for col in data.columns if y_week in col]
            columns_y  = [geography] + weeks_y
            data_y = data[columns_y]
            ### now add the final column to the y data that has it so that it's if any week in the trhee week perdiod exceeded 15
            train_week = w2n.word_to_num(x_week.replace("_", ""))
            target_week =  w2n.word_to_num(y_week.replace("_", ""))
            y_weeks_to_check = []
            for week_to_check in range(train_week + 1, target_week + 1):
                y_weeks_to_check.append('_' + num2words(week_to_check) + '_')

            y_weeks_to_check = [week + 'beds_over_15_100k' for week in y_weeks_to_check]
            columns_to_check = [col for col in data.columns if any(week in col for week in y_weeks_to_check)]
            y_over_in_period = data[columns_to_check].apply(max, axis=1)
            data_y = pd.concat([data_y, y_over_in_period], axis=1)
            # ensure they have the same amount of data
            #remove rows in test_data1 with NA in test_data2
            data_x = data_x.dropna()
            data_x = data_x[data_x[geography].isin(data_y[geography])]
            # remove rows in test_data2 with NA in test_data1
            data_y = data_y.dropna()
            data_y = data_y[data_y[geography].isin(data_x[geography])]
            data_x = data_x[data_x[geography].isin(data_y[geography])]
            data_x_no_HSA = len(data_x[geography].unique())

            missing_data.append(((len(data[geography].unique()) - data_x_no_HSA)/len(data[geography].unique())) * 100)
            # get weights 
            #weights = weight_data[weight_data[geography].isin(data_x[geography])][[geography, weight_col]]

            X_week = data_x.iloc[:, 1:len(columns_x)]  # take away y, leave weights for mo
            y_week = data_y.iloc[:, -1] 
            
            y_week = y_week.astype(int)
            if if_train:

                 X_week, y_week = oversample.fit_resample(X_week, y_week)
            weights = X_week.iloc[:, -1] 
            X_week = X_week.iloc[:, :len(X_week.columns)-1] # remove the weights and leave "target" for that week, inlcuding it as a feature

            #rename columns for concatenation 
            X_week.columns = range(1, len(data_x.columns) -1)
            y_week.columns = range(1, len(data_y.columns) -1)
            X_data = pd.concat([X_data, X_week])
            y_data = pd.concat([y_data, y_week]) 
        
            weights_all =  pd.concat([weights_all, weights]) 


    X_data.reset_index(drop=True, inplace=True)
    y_data.reset_index(drop=True, inplace=True)
    weights_all.reset_index(drop=True, inplace=True)

    return(X_data, y_data, weights_all, missing_data)

Data prep
For the CDC, they only use three metrics: new_cases_per_100k, new_admits_per_100k, percent_beds_100k

In [ ]:
data_by_HSA = pd.read_csv('/Users/rem76/Documents/COVID_projections/hsa_time_data_all_dates.csv')
data_by_HSA['health_service_area_number']
data_by_HSA['health_service_area']
#data_by_HSA['HSA_ID'] = data_by_HSA['health_service_area_number'].astype(str) + '' + data_by_HSA['health_service_area'].apply(lambda x: x.split()[0])
data_by_HSA.rename(columns={'health_service_area_number': 'HSA_ID'}, inplace=True)
data_by_HSA.columns

In [81]:
data_by_HSA = pd.read_csv('/Users/rem76/Documents/COVID_projections/hsa_time_data_all_dates.csv')
data_by_HSA['health_service_area_number']
data_by_HSA['health_service_area']
#data_by_HSA['HSA_ID'] = data_by_HSA['health_service_area_number'].astype(str) + '' + data_by_HSA['health_service_area'].apply(lambda x: x.split()[0])
data_by_HSA.rename(columns={'health_service_area_number': 'HSA_ID'}, inplace=True)

data_by_HSA['beds_over_15_100k'] = (data_by_HSA['beds_weekly'] > 15)*1

# remove HSAs that have missing data in specific columns

data_by_HSA = data_by_HSA.dropna(subset=['admits_weekly', 'cases_weekly', 'perc_covid'])
for i, week in enumerate(data_by_HSA['date'].unique()):
    data_by_HSA.loc[data_by_HSA['date'] == week, 'week'] = i
    
data_by_HSA_cases = data_by_HSA[['HSA_ID', 'week', 'cases_weekly']]
data_by_HSA_cases = data_by_HSA_cases.pivot_table(index= 'week', columns='HSA_ID', values='cases_weekly')

data_by_HSA_admissions = data_by_HSA[['HSA_ID', 'week', 'admits_weekly']]
data_by_HSA_admissions = data_by_HSA_admissions.pivot_table(index= 'week', columns='HSA_ID', values='admits_weekly')

data_by_HSA_perc_covid = data_by_HSA[['HSA_ID', 'week', 'perc_covid']]
data_by_HSA_perc_covid = data_by_HSA_perc_covid.pivot_table(index= 'week', columns='HSA_ID', values='perc_covid')



data_data_cases_admits_weekly = pd.merge(data_by_HSA_cases, data_by_HSA_admissions, on='week')
new_column_names = [col.replace('_x', '_cases') for col in data_data_cases_admits_weekly.columns]
data_data_cases_admits_weekly.rename(columns=dict(zip(data_data_cases_admits_weekly.columns, new_column_names)), inplace=True)
new_column_names = [col.replace('_y', '_admits') for col in data_data_cases_admits_weekly.columns]
data_data_cases_admits_weekly.rename(columns=dict(zip(data_data_cases_admits_weekly.columns, new_column_names)), inplace=True)

old_column_names = data_by_HSA_perc_covid.columns
new_column_names = [str(col) + '_percent_beds_covid' for col in old_column_names]
new_column_names = dict(zip(old_column_names, new_column_names))
data_by_HSA_perc_covid.rename(columns=new_column_names, inplace=True)

data_by_HSA_cases_admits_perc = pd.merge(data_data_cases_admits_weekly, data_by_HSA_perc_covid, on='week')
data_by_HSA_cases_admits_perc = data_by_HSA_cases_admits_perc.reset_index()
#data_by_HSA_cases_admits_perc.columns = data_by_HSA_cases_admits_perc.columns.str.replace(',', '')

/var/folders/d4/chxwf4hs5kq7ttsp56s64z65mjk3qj/T/ipykernel_58265/1329533506.py:1: DtypeWarning: Columns (41,43,44,45,46,50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  data_by_HSA = pd.read_csv('/Users/rem76/Documents/COVID_projections/hsa_time_data_all_dates.csv')


In [107]:
column_names = ['HSA_ID']
categories_for_subsetting = ['cases', 'admits', 'percent_beds_covid']

for week in range(1, len(data_by_HSA_cases_admits_perc) + 1):
    week = num2words(week)
    for category in categories_for_subsetting:
        column_name = f'week_{week}_{category}'
        column_names.append(column_name)

all_HSA_ID_weekly_data = pd.DataFrame(index=range(51), columns=column_names)

x = 0

for HSA in data_by_HSA['HSA_ID'].unique():
    matching_indices = [i for i, HSA_col in enumerate(data_by_HSA_cases_admits_perc) if HSA_col == HSA]
    all_HSA_ID_weekly_data.loc[x, 'HSA_ID'] = HSA
    columns_to_subset =  [f'{HSA}_{category}' for category in categories_for_subsetting]
 
    j = 1
    try:
        for row in range(len(data_by_HSA_cases_admits_perc.loc[:, columns_to_subset])):
            all_HSA_ID_weekly_data.iloc[x, j:j + len(categories_for_subsetting)] = data_by_HSA_cases_admits_perc.loc[row,columns_to_subset]
            j += 3
    except:
        pass
    x += 1



weights_df = data_by_HSA[data_by_HSA['HSA_ID'].isin(all_HSA_ID_weekly_data['HSA_ID'])][['HSA_ID','weight_alt']]
weights_df = weights_df.rename(columns = {'HSA_ID': 'HSA_ID', 'weight_alt':'weight'})
weights_df = weights_df.drop_duplicates()
weights_df['weight'].unique()
all_HSA_ID_weekly_data = all_HSA_ID_weekly_data.join(weights_df['weight'])

In [83]:
X_train, y_train, weights, missing_data_train_HSA = prep_training_test_data_period(all_HSA_ID_weekly_data, [3,2], no_weeks = range(1, int(123*2/3) + 1), weeks_in_futre = 3, if_train = False, geography = 'HSA_ID', weight_col = 'weight')

X_test, y_test, weights_test, missing_data_test_HSA = prep_training_test_data_period(all_HSA_ID_weekly_data, [3,2], no_weeks = range(int(123*2/3) + 1, 123), weeks_in_futre = 3, if_train = False, geography = 'HSA_ID',  weight_col = 'weight')
weights = weights[0].to_numpy()


In [84]:
def determine_covid_outcome(new_cases_per_100k, new_admits_per_100k, percent_beds_100k):

    ## here we're saying that High = >15 per 100k
    if new_cases_per_100k < 200:
        if (new_admits_per_100k >= 10) | (percent_beds_100k > 10):
            if (new_admits_per_100k >= 20) | (percent_beds_100k >= 15):
                #return "High"
                return 1
            else:
                return 0
                #return "Medium"
        else:
                return 0
            #return "Low"
    elif 200 <= new_cases_per_100k:
        if (new_admits_per_100k >= 10) | (percent_beds_100k > 10):
            if (new_admits_per_100k >= 10) | (percent_beds_100k >= 10):
                #return "High"
                return 1
            else:
                return 0
                #return "Medium"


Just compare the High vs >15 per 100k in the week of interest (no time lag)

In [85]:
# Assuming you have a list called 'categories_for_subsetting' that contains the categories you want to subset
CDC_predictions = pd.DataFrame(index=data_by_HSA_cases_admits_perc['week'])

for HSA in data_by_HSA['HSA_ID'].unique():
    matching_indices = [i for i, HSA_col in enumerate(data_by_HSA_cases_admits_perc.columns) if HSA_col == HSA]
    columns_to_subset = [f'{HSA}_{category}' for category in categories_for_subsetting]
    j = 0
    try:
        for idx, row in data_by_HSA_cases_admits_perc.iterrows():
            # Pass each column from 'columns_to_subset' into the 'determine_covid_outcome' function
            outcome = determine_covid_outcome(row[columns_to_subset[0]], row[columns_to_subset[1]], row[columns_to_subset[2]])
            CDC_predictions.loc[idx, f'{HSA}_outcome'] = outcome
    except Exception as e:
        print(f"Error occurred for HSA {HSA}: {e}")

/var/folders/d4/chxwf4hs5kq7ttsp56s64z65mjk3qj/T/ipykernel_58265/1452187041.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  CDC_predictions.loc[idx, f'{HSA}_outcome'] = outcome
/var/folders/d4/chxwf4hs5kq7ttsp56s64z65mjk3qj/T/ipykernel_58265/1452187041.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  CDC_predictions.loc[idx, f'{HSA}_outcome'] = outcome
/var/folders/d4/chxwf4hs5kq7ttsp56s64z65mjk3qj/T/ipykernel_58265/1452187041.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of call

In [88]:
data_by_HSA_over_15_100k = data_by_HSA[['HSA_ID', 'week', 'beds_over_15_100k']]
data_by_HSA_over_15_100k = data_by_HSA_over_15_100k.pivot_table(index= 'week', columns='HSA_ID', values='beds_over_15_100k')

old_column_names = data_by_HSA_over_15_100k.columns
new_column_names = [str(col) + '_beds_over_15_100k' for col in old_column_names]
new_column_names = dict(zip(old_column_names, new_column_names))
data_by_HSA_over_15_100k.rename(columns=new_column_names, inplace=True)
data_by_HSA_over_15_100k = data_by_HSA_over_15_100k.reset_index()
actual_hospital_capacity = data_by_HSA_over_15_100k.filter(like="beds_over_15_100k")

# Create a new DataFrame with the subset columns
actual_hospital_capacity = pd.DataFrame(actual_hospital_capacity)


In [97]:
actual_hospital_capacity.fillna(0, inplace=True)
CDC_predictions.fillna(0, inplace=True)


In [98]:
comparison_result = actual_hospital_capacity.values == CDC_predictions.values
correct_predictions = comparison_result.sum()

# Calculate the total number of predictions (total number of elements in the DataFrames)
total_predictions = actual_hospital_capacity.size
accuracy = correct_predictions / total_predictions
accuracy

0.5635678198156504

In [102]:
# Convert predicted probabilities to binary predictions using a threshold (e.g., 0.5)

# Calculate AUROC
auroc = roc_auc_score(actual_hospital_capacity.values.ravel(), CDC_predictions.values.ravel())

# Calculate accuracy
accuracy = accuracy_score(actual_hospital_capacity.values.ravel(), CDC_predictions.values.ravel())

# Calculate confusion matrix
conf_matrix = confusion_matrix(actual_hospital_capacity.values.ravel(), CDC_predictions.values.ravel())

# Calculate sensitivity (True Positive Rate)
sensitivity = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])

# Calculate specificity (True Negative Rate)
specificity = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])

# Calculate PPV (Positive Predictive Value)
ppv = conf_matrix[1, 1] / (conf_matrix[0, 1] + conf_matrix[1, 1])

# Calculate NPV (Negative Predictive Value)
npv = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[1, 0])


In [104]:
print("sensitivity:", sensitivity)
print("specificity:", specificity)
print("accuracy:", accuracy)
print("auroc:", auroc)

print("ppv:", ppv)
print("npv:", npv)

sensitivity: 0.42439316595478693
specificity: 0.9605184965086911
accuracy: 0.5635678198156504
auroc: 0.692455831231739
ppv: 0.9684129200962767
npv: 0.3691106575510612


Now look at t + 3 weeks, with the CDC community levels as the feature and whether the capacity execeeds 15 per 100k in the next 3 weeks as the outcome

In [109]:
def determine_covid_outcome(new_cases_per_100k, new_admits_per_100k, percent_beds_100k):

    ## here we're saying that High = >15 per 100k
    if new_cases_per_100k < 200:
        if (new_admits_per_100k >= 10) | (percent_beds_100k > 10):
            if (new_admits_per_100k >= 20) | (percent_beds_100k >= 15):
                return "High"
            else:
                return "Medium"
        else:
            return "Low"
    elif 200 <= new_cases_per_100k:
        if (new_admits_per_100k >= 10) | (percent_beds_100k > 10):
            if (new_admits_per_100k >= 10) | (percent_beds_100k >= 10):
                return "High"
            else:
                return "Medium"

1. CDC predictions

In [132]:
data_by_HSA = pd.read_csv('/Users/rem76/Documents/COVID_projections/hsa_time_data_all_dates.csv')
data_by_HSA['health_service_area_number']
data_by_HSA['health_service_area']
#data_by_HSA['HSA_ID'] = data_by_HSA['health_service_area_number'].astype(str) + '' + data_by_HSA['health_service_area'].apply(lambda x: x.split()[0])
data_by_HSA.rename(columns={'health_service_area_number': 'HSA_ID'}, inplace=True)

data_by_HSA['beds_over_15_100k'] = (data_by_HSA['beds_weekly'] > 15)*1

# remove HSAs that have missing data in specific columns

data_by_HSA = data_by_HSA.dropna(subset=['admits_weekly', 'cases_weekly', 'perc_covid'])
for i, week in enumerate(data_by_HSA['date'].unique()):
    data_by_HSA.loc[data_by_HSA['date'] == week, 'week'] = i
    
data_by_HSA_cases = data_by_HSA[['HSA_ID', 'week', 'cases_weekly']]
data_by_HSA_cases = data_by_HSA_cases.pivot_table(index= 'week', columns='HSA_ID', values='cases_weekly')

data_by_HSA_admissions = data_by_HSA[['HSA_ID', 'week', 'admits_weekly']]
data_by_HSA_admissions = data_by_HSA_admissions.pivot_table(index= 'week', columns='HSA_ID', values='admits_weekly')

data_by_HSA_perc_covid = data_by_HSA[['HSA_ID', 'week', 'perc_covid']]
data_by_HSA_perc_covid = data_by_HSA_perc_covid.pivot_table(index= 'week', columns='HSA_ID', values='perc_covid')



data_data_cases_admits_weekly = pd.merge(data_by_HSA_cases, data_by_HSA_admissions, on='week')
new_column_names = [col.replace('_x', '_cases') for col in data_data_cases_admits_weekly.columns]
data_data_cases_admits_weekly.rename(columns=dict(zip(data_data_cases_admits_weekly.columns, new_column_names)), inplace=True)
new_column_names = [col.replace('_y', '_admits') for col in data_data_cases_admits_weekly.columns]
data_data_cases_admits_weekly.rename(columns=dict(zip(data_data_cases_admits_weekly.columns, new_column_names)), inplace=True)

old_column_names = data_by_HSA_perc_covid.columns
new_column_names = [str(col) + '_percent_beds_covid' for col in old_column_names]
new_column_names = dict(zip(old_column_names, new_column_names))
data_by_HSA_perc_covid.rename(columns=new_column_names, inplace=True)

data_by_HSA_cases_admits_perc = pd.merge(data_data_cases_admits_weekly, data_by_HSA_perc_covid, on='week')
data_by_HSA_cases_admits_perc = data_by_HSA_cases_admits_perc.reset_index()
#data_by_HSA_cases_admits_perc.columns = data_by_HSA_cases_admits_perc.columns.str.replace(',', '')

/var/folders/d4/chxwf4hs5kq7ttsp56s64z65mjk3qj/T/ipykernel_58265/1329533506.py:1: DtypeWarning: Columns (41,43,44,45,46,50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  data_by_HSA = pd.read_csv('/Users/rem76/Documents/COVID_projections/hsa_time_data_all_dates.csv')


In [ ]:
# Assuming you have a list called 'categories_for_subsetting' that contains the categories you want to subset
CDC_predictions = pd.DataFrame(index=data_by_HSA_cases_admits_perc['week'])
categories_for_subsetting = ['cases', 'admits', 'percent_beds_covid']
for HSA in data_by_HSA['HSA_ID'].unique():
    matching_indices = [i for i, HSA_col in enumerate(data_by_HSA_cases_admits_perc.columns) if HSA_col == HSA]
    columns_to_subset = [f'{HSA}_{category}' for category in categories_for_subsetting]
    j = 0
    try:
        for idx, row in data_by_HSA_cases_admits_perc.iterrows():
            # Pass each column from 'columns_to_subset' into the 'determine_covid_outcome' function
            outcome = determine_covid_outcome(row[columns_to_subset[0]], row[columns_to_subset[1]], row[columns_to_subset[2]])
            CDC_predictions.loc[idx, f'{HSA}_category'] = outcome
    except Exception as e:
        print(f"Error occurred for HSA {HSA}: {e}")

In [137]:
column_names = ['HSA_ID']
categories_for_subsetting = ['category']

for week in range(1, len(CDC_predictions) + 1):
    week = num2words(week)
    for category in categories_for_subsetting:
        column_name = f'week_{week}_{category}'
        column_names.append(column_name)

CDC_predictions_by_week = pd.DataFrame(index=range(51), columns=column_names)

x = 0

categories_for_subsetting = ['category']
for HSA in data_by_HSA['HSA_ID'].unique():
    matching_indices = [i for i, HSA_col in enumerate(CDC_predictions) if HSA_col == HSA]
    CDC_predictions_by_week.loc[x, 'HSA_ID'] = HSA
    columns_to_subset =  [f'{HSA}_{category}' for category in categories_for_subsetting]
 
    j = 1
    try:
        for row in range(len(CDC_predictions.loc[:, columns_to_subset])):
            CDC_predictions_by_week.iloc[x, j:j + len(categories_for_subsetting)] = CDC_predictions.loc[row,columns_to_subset]
            j += 1
    except:
        pass
    x += 1

2. Outcomes: over/under 15 per 100k 

In [142]:
data_by_HSA_over_15_100k = data_by_HSA[['HSA_ID', 'week', 'beds_over_15_100k']]
data_by_HSA_over_15_100k = data_by_HSA_over_15_100k.pivot_table(index= 'week', columns='HSA_ID', values='beds_over_15_100k')

old_column_names = data_by_HSA_over_15_100k.columns
new_column_names = [str(col) + '_beds_over_15_100k' for col in old_column_names]
new_column_names = dict(zip(old_column_names, new_column_names))
data_by_HSA_over_15_100k.rename(columns=new_column_names, inplace=True)
data_by_HSA_over_15_100k = data_by_HSA_over_15_100k.reset_index()
column_names = ['HSA_ID']
categories_for_subsetting = ['beds_over_15_100k']

for week in range(1, len(data_by_HSA_over_15_100k) + 1):
    week = num2words(week)
    for category in categories_for_subsetting:
        column_name = f'week_{week}_{category}'
        column_names.append(column_name)

all_HSA_ID_weekly_data = pd.DataFrame(index=range(len(data_by_HSA['HSA_ID'].unique())), columns=column_names)

x = 0

categories_for_subsetting = ['beds_over_15_100k']
for HSA in data_by_HSA['HSA_ID'].unique():
    matching_indices = [i for i, HSA_col in enumerate(data_by_HSA_over_15_100k) if HSA_col == HSA]
    all_HSA_ID_weekly_data.loc[x, 'HSA_ID'] = HSA
    columns_to_subset =  tuple([f'{HSA}_{category}' for category in categories_for_subsetting]) 
    #columns_to_subset =  [f'{HSA}_{category}' for category in categories_for_subsetting]
 
    j = 1
    try:
        for row in range(len(data_by_HSA_over_15_100k.loc[:, columns_to_subset])):
            all_HSA_ID_weekly_data.iloc[x, j:j + len(categories_for_subsetting)] = data_by_HSA_over_15_100k.loc[row,columns_to_subset]
            j += 1
    except:
        pass
    x += 1

In [125]:
all_HSA_ID_weekly_data

,HSA_ID,week_one_beds_over_15_100k,week_two_beds_over_15_100k,week_three_beds_over_15_100k,week_four_beds_over_15_100k,week_five_beds_over_15_100k,week_six_beds_over_15_100k,week_seven_beds_over_15_100k,week_eight_beds_over_15_100k,week_nine_beds_over_15_100k,...,week_one hundred and twenty_beds_over_15_100k,week_one hundred and twenty-one_beds_over_15_100k,week_one hundred and twenty-two_beds_over_15_100k,week_one hundred and twenty-three_beds_over_15_100k,week_one hundred and twenty-four_beds_over_15_100k,week_one hundred and twenty-five_beds_over_15_100k,week_one hundred and twenty-six_beds_over_15_100k,week_one hundred and twenty-seven_beds_over_15_100k,week_one hundred and twenty-eight_beds_over_15_100k,week_one hundred and twenty-nine_beds_over_15_100k
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
800,822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
801,823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
802,904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
